In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator
import theano
import theano.tensor as T
import csv

In [2]:
#generate the file we will be using
def generateDataFiles(plot = True, size = 0, w1 = 0, w2 = 0):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)
    epsilon = 0.05

    data = open("Data.txt", "w")
    writer = csv.writer(data, delimiter=',')
    
    if size == 0:
        size = 50

    x1 = np.random.uniform(-10,10,size)
    x2 = np.random.uniform(-10,10,size)

    if w1 == 0:
        w1 = np.random.uniform(-2,2,1)[0]
    if w2 == 0:
        w2 = np.random.uniform(-2,2,1)[0]
    print("Actual w1 = %f and w2 = %f\n"%(w1,w2))

    for i in range(0,size):
        val1 = x1[i]
        val2 = x2[i]
        k = logistic([val1,val2],[w1,w2])
        if 1 - k < epsilon:
            writer.writerow([1,val1,val2])
        elif k < epsilon:
            writer.writerow([0,val1,val2])
    data.close()
    if plot:
        plotData(w1,w2)


In [3]:
#calculate Error from training data
def calculateError(w1, w2, display = True):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)
    
    if display:
        print("With: w1 = %f and w2 = %f"%(w1,w2))
    

    data = []
    sumError = 0
    sumErrorSquare = 0
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    if display:
        print('y\t\tf(x)\t\tE\t\tE^2')
    for i in range(0,len(data)):
        x1 = data[i][1]
        x2 = data[i][2]
        f = logistic([x1,x2],[w1,w2])
        e = data[i][0] - f
        e2 = e**2
        sumError += e
        sumErrorSquare += e2
        if display:
            print('%f\t%f\t%f\t%f'%(data[i][0],f,e,e2))
    if display:
        print("\nSum:\t\t\t\t%f\t%f"%(sumError, sumErrorSquare))
    return sumErrorSquare

In [4]:
#plot Data from Data.txt
def plotData(w1,w2):
    data = []
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    for i in range(0,len(data)):
        if data[i][0] == 1:
            plt.scatter(data[i][1],data[i][2],20, 'r')
        else:
            plt.scatter(data[i][1],data[i][2],20, 'y')
    plt.plot([0,w1],[0,w2])
    plt.show()

In [36]:
#attempt to find my w
def findW(plot = True, showError = True, epsilon = 0.1):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)

    gs = T.grad(s, w)
    dlogistic = theano.function([x,w], gs)
    

    w1 = np.random.uniform(-2,2,1)[0]
    w2 = np.random.uniform(-2,2,1)[0]
    print("Starting Values: w1 = %f and w2 = %f"%(w1,w2))

    data = []
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    sumErrorSquare = 1
    iterations = 0
    while sumErrorSquare > epsilon:
        sumError = 0
        sumErrorSquare = 0
        sumErrorDeriv = 0
        for i in range(0,len(data)):
            x1 = data[i][1]
            x2 = data[i][2]
            f = logistic([x1,x2],[w1,w2])
            e = data[i][0] - f
            e2 = e**2
            sumError += e
            sumErrorSquare = sumErrorSquare + e2
            sumErrorDeriv -= 2 * e * dlogistic([x1,x2],[w1,w2])
        if iterations > 200:
            print("")
        w1 = w1 -  0.1 * sumErrorDeriv[0]
        w2 = w2 - 0.1 * sumErrorDeriv[1]
    print("Final Values: w1 = %f and w2 = %f"%(w1,w2))
    print("iterations took:",iterations)
    if plot:
        plotData(w1,w2)
    if showError:    
        print("\nCalculating Error ... ")
    return calculateError(w1,w2, showError)
    

In [37]:
#testing the the function to find a w
generateDataFiles(False)
for i in range(0,4):
    error = findW(False, False)
    print("The error for these values:",error)
    print("")
    


Actual w1 = 1.845551 and w2 = -1.008407

Starting Values: w1 = -0.187673 and w2 = -1.779409
Final Values: w1 = 1.953915 and w2 = -0.976632
iterations took: 147
The error for these values: 0.00099150943416

Starting Values: w1 = 0.362837 and w2 = 1.377963
Final Values: w1 = 2.103517 and w2 = -0.675106
iterations took: 33
The error for these values: 0.000976116373943

Starting Values: w1 = 1.766126 and w2 = -1.319515
Final Values: w1 = 2.028401 and w2 = -1.087431
iterations took: 132
The error for these values: 0.000994077438104

Starting Values: w1 = -1.590134 and w2 = -1.448304
Final Values: w1 = 2.309835 and w2 = -1.450566
iterations took: 73
The error for these values: 0.000982798992901



In [6]:

def generateAndFindWPlotError():
    data = generateGraphData(20)
    print("Done Generating Grid!")
    xHolder = data[0]
    yHolder = data[1]
    z = np.array([calculateError(x,y, False) for x,y in zip(np.ravel(xHolder), np.ravel(yHolder))])
    print("Done Calculating Errors")
    Z = z.reshape(xHolder.shape)
    data.extend([Z])
    return data

In [ ]:
#testing functions to plot data
data = generateAndFindWPlotError()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(data[0], data[1],data[2],rstride=1, cstride=1, cmap=cm.coolwarm)

ax.set_xlabel('W1 Axis')
ax.set_ylabel('W2 Axis')
ax.set_zlabel('Error Axis')

plt.show()

Done Generating Grid!
Done Calculating Errors

In [7]:
#used to genereate range of values for w1 and w2
def generateGraphData(numberOfEach):
    x = np.linspace(-10,10,numberOfEach)
    #x = np.repeat(x, numberOfEach)
    y = np.linspace(-10,10,numberOfEach)
    #y = np.tile(y,numberOfEach)
    x, y = np.meshgrid(x,y)
    return [x,y]

In [28]:
#testing validity of the generateAndFindWPlotError function and generateGraphData

data = generateAndFindWPlotError()
goodData = []
for x,y,z in zip(np.ravel(data[0]), np.ravel(data[1]), np.ravel(data[2])):
    if z < 0.01:
        goodData.extend([[x, y]])
        plotData(x,y)
print(goodData)

Done Generating Grid!
Done Calculating Errors
[[0.0, 1.5], [-1.5, 3.0], [0.0, 3.0]]
